In [1]:
!pip uninstall keras
!pip install keras==2.3.1

Found existing installation: Keras 2.3.1
Uninstalling Keras-2.3.1:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/Keras-2.3.1.dist-info/*
    /usr/local/lib/python3.7/dist-packages/docs/*
    /usr/local/lib/python3.7/dist-packages/keras/*
  Would not remove (might be manually added):
    /usr/local/lib/python3.7/dist-packages/docs/md_autogen.py
    /usr/local/lib/python3.7/dist-packages/docs/update_docs.py
Proceed (y/n)? y
  Successfully uninstalled Keras-2.3.1
  Using cached Keras-2.3.1-py2.py3-none-any.whl (377 kB)


In [2]:
!pip uninstall tensorflow
!pip install tensorflow==2.1.0

Found existing installation: tensorflow 2.1.0
Uninstalling tensorflow-2.1.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.7/dist-packages/tensorflow-2.1.0.dist-info/*
    /usr/local/lib/python3.7/dist-packages/tensorflow/*
    /usr/local/lib/python3.7/dist-packages/tensorflow_core/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-2.1.0
  Using cached tensorflow-2.1.0-cp37-cp37m-manylinux2010_x86_64.whl (421.8 MB)


In [3]:
!pip install 'h5py==2.10.0' --force-reinstall

  Using cached h5py-2.10.0-cp37-cp37m-manylinux1_x86_64.whl (2.9 MB)
  Using cached six-1.16.0-py2.py3-none-any.whl (11 kB)
  Using cached numpy-1.21.4-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (15.7 MB)
  Attempting uninstall: six
    Found existing installation: six 1.16.0
    Uninstalling six-1.16.0:
      Successfully uninstalled six-1.16.0
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.4
    Uninstalling numpy-1.21.4:
      Successfully uninstalled numpy-1.21.4
  Attempting uninstall: h5py
    Found existing installation: h5py 2.10.0
    Uninstalling h5py-2.10.0:
      Successfully uninstalled h5py-2.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yellowbrick 1.3.post1 requires numpy<1.20,>=1.16.0, but you have numpy 1.21.4 which is incompatible.
tensorflow-probability 0.15.0 requires gast>=0.3.2, but you hav

In [4]:
!pip install matchzoo

In [5]:
### INIT ###

import keras
import pandas as pd
import numpy as np
import matchzoo as mz
import json

print('data loading ...')
train_pack_raw = mz.datasets.wiki_qa.load_data('train', task='ranking')
dev_pack_raw = mz.datasets.wiki_qa.load_data('dev', task='ranking', filtered=True)
test_pack_raw = mz.datasets.wiki_qa.load_data('test', task='ranking', filtered=True)
print('data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`')

ranking_task = mz.tasks.Ranking(loss=mz.losses.RankHingeLoss())
ranking_task.metrics = [
    mz.metrics.NormalizedDiscountedCumulativeGain(k=3),
    mz.metrics.NormalizedDiscountedCumulativeGain(k=5),
    mz.metrics.MeanAveragePrecision(),
    mz.metrics.AveragePrecision(),
    mz.metrics.MeanReciprocalRank(),
    mz.metrics.Precision()
]
print("`ranking_task` initialized with metrics", ranking_task.metrics)

print("loading embedding ...")
glove_embedding = mz.datasets.embeddings.load_glove_embedding(dimension=300)
print("embedding loaded as `glove_embedding`")

Using TensorFlow backend.


data loading ...
data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`
`ranking_task` initialized with metrics [normalized_discounted_cumulative_gain@3(0.0), normalized_discounted_cumulative_gain@5(0.0), mean_average_precision(0.0), average_precision(0.0), mean_reciprocal_rank(0.0), precision@1(0.0)]
loading embedding ...
embedding loaded as `glove_embedding`


In [6]:
def append_params_to_readme(model):
    import tabulate
    
    with open('README.rst', 'a+') as f:
        subtitle = model.params['model_class'].__name__
        line = '#' * len(subtitle)
        subtitle = subtitle + '\n' + line + '\n\n'
        f.write(subtitle)
        
        df = model.params.to_frame()[['Name', 'Value']]
        table = tabulate.tabulate(df, tablefmt='rst', headers='keys') + '\n\n'
        f.write(table)

  ### END INIT ### 

In [7]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [8]:
### KRNM ###

preprocessor = mz.preprocessors.BasicPreprocessor(fixed_length_left=10, fixed_length_right=40, remove_stop_words=False)
train_pack_processed = preprocessor.fit_transform(train_pack_raw)
valid_pack_processed = preprocessor.transform(dev_pack_raw)
test_pack_processed = preprocessor.transform(test_pack_raw)

Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 9169.41it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:03<00:00, 4888.78it/s]
Building Vocabulary from a datapack.: 100%|██████████| 473697/473697 [00:00<00:00, 3146475.27it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 9533.53it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:03<00:00, 5103.78it/s]
Processing text_right with transform: 100%|██████████| 18841/18841 [00:00<00:00, 95996.37it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 122/122 [00:00<00:00, 8497.97it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 1115/1115 [00:00

In [9]:
preprocessor.context

{'embedding_input_dim': 17483,
 'filter_unit': <matchzoo.preprocessors.units.frequency_filter.FrequencyFilter at 0x7f9602082650>,
 'input_shapes': [(10,), (40,)],
 'vocab_size': 17483,
 'vocab_unit': <matchzoo.preprocessors.units.vocabulary.Vocabulary at 0x7f95fe1c5590>}

In [10]:
model = mz.models.KNRM()
model.params.update(preprocessor.context)
model.params['task'] = ranking_task
model.params['embedding_output_dim'] = glove_embedding.output_dim
model.params['embedding_trainable'] = True
model.params['kernel_num'] = 21
model.params['sigma'] = 0.1
model.params['exact_sigma'] = 0.001
model.params['optimizer'] = 'adadelta'
model.build()
model.compile()

In [11]:
embedding_matrix = glove_embedding.build_matrix(preprocessor.context['vocab_unit'].state['term_index'])
model.load_embedding_matrix(embedding_matrix)

In [12]:
pred_x, pred_y = test_pack_processed.unpack()
evaluate = mz.callbacks.EvaluateAllMetrics(model, x=pred_x, y=pred_y, batch_size=len(pred_x), model_save_path='./krnm_pretrained_model/')

In [13]:
train_generator = mz.DataGenerator(
    train_pack_processed,
    mode='pair',
    num_dup=5,
    num_neg=1,
    batch_size=20
)
print('num batches:', len(train_generator))

num batches: 255


In [14]:
%%time
history = model.fit_generator(train_generator, epochs=15, callbacks=[evaluate], workers=30, use_multiprocessing=True)

/usr/local/lib/python3.7/dist-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/15
255/255 [==============================] - 16s 64ms/step - loss: 0.9873
Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5132428318135231 - normalized_discounted_cumulative_gain@5(0.0): 0.5804449225019014 - mean_average_precision(0.0): 0.5409701368375979 - average_precision(0.0): 0.2573661062519775 - mean_reciprocal_rank(0.0): 0.550299871800244 - precision@1(0.0): 0.3755274261603376
Epoch 2/15
255/255 [==============================] - 14s 55ms/step - loss: 0.4802
Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5330521466876161 - normalized_discounted_cumulative_gain@5(0.0): 0.5920466806096554 - mean_average_precision(0.0): 0.5512794298159445 - average_precision(0.0): 0.25930206621232743 - mean_reciprocal_rank(0.0): 0.564477687655275 - precision@1(0.0): 0.379746835443038
Epoch 3/15
255/255 [==============================] - 16s 64ms/step - loss: 0.3508
Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5306581424621547 - normalized_discounted_

In [15]:
model.evaluate(pred_x, pred_y)
model.predict(pred_x)

array([[ 0.48412874],
       [-1.5032939 ],
       [-0.19357382],
       ...,
       [-0.08265347],
       [ 0.7505069 ],
       [ 1.7492442 ]], dtype=float32)